In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/classification_of_reviews/wb_reviews.csv"

data = pd.read_csv(file_path)

Mounted at /content/drive


In [3]:
data['Rating'] = data['Rating'].apply(lambda x: x*2)
data['Sentiment'] = data['Rating'].apply(lambda x: 1 if x >= 7 else 0)
data = data.dropna()

In [4]:
data

,Good's name,Description,Review,Rating,Sentiment
0,Мочалка для душа Grizzly XXL для тела черная,Готовы к незабываемым впечатлениям в душе? «Gr...,Хорошо моет,10,1
1,Мочалка для душа Grizzly XXL для тела черная,Готовы к незабываемым впечатлениям в душе? «Gr...,"При первом использовании заметил ,что мочалка ...",2,0
2,Мочалка для душа Grizzly XXL для тела черная,Готовы к незабываемым впечатлениям в душе? «Gr...,Супер всё!,10,1
3,Мочалка для душа Grizzly XXL для тела черная,Готовы к незабываемым впечатлениям в душе? «Gr...,"Пошита, на первый взгляд, не плохо. Мылится не...",10,1
4,Мочалка для душа Grizzly XXL для тела черная,Готовы к незабываемым впечатлениям в душе? «Gr...,Слишком мягкая,2,0
...,...,...,...,...,...
196940,Звонок дверной беспроводной на батарейках,Звонок дверной беспроводной влагозащищенный ул...,Прекрасный звонок. У нас получилась дальность ...,10,1
196941,Звонок дверной беспроводной на батарейках,Звонок дверной беспроводной влагозащищенный ул...,Всё супер. Звонок просто ВАУУУ,10,1
196942,Звонок дверной беспроводной на батарейках,Звонок дверной беспроводной влагозащищенный ул...,"Классный звонок, сколько у нас из было,этот са...",10,1
196943,Звонок дверной беспроводной на батарейках,Звонок дверной беспроводной влагозащищенный ул...,"Звенит звонко Нет Очень хороший звоночек, мн...",10,1


## Предобработка данных

In [5]:
data_clean = data.copy()

In [6]:
nltk.download('punkt')
nltk.download('punkt_tab')

stemmer = SnowballStemmer("russian")

def count_russian_chars(s: str) -> int:
    return len(re.findall(r'[а-яА-Я]', s))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    words = word_tokenize(text, language="russian")
    words = [stemmer.stem(word) for word in words]
    return " ".join(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
data_clean['Russian chars'] = data_clean['Review'].apply(count_russian_chars)
data_clean_russian = data_clean[data_clean['Russian chars'] > 0]
data_clean_russian = data_clean_russian.reset_index(drop=True)
data_clean_russian['Review'] = data_clean_russian['Review'].apply(lambda x: preprocess_text(str(x)))

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_clean_russian[['Review']],
                                                    data_clean_russian['Sentiment'],
                                                    stratify=data_clean_russian['Sentiment'],
                                                    test_size=0.25,
                                                    random_state=42)

## Обучение лучшей модели бейзлайна

Логистическая регрессия с базовой предобработкой (стемминг) и подбором гиперпараметров

In [10]:
pipeline = Pipeline([
    ('tf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LogisticRegression(random_state=42, C=10))
])

param_grid = {
    # Уже подбирали в соответствующем ноутбуке (wb_baseline, пункт 3)
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision', n_jobs=-1)

In [11]:
grid_search.fit(X_train['Review'], y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tf',
                                        TfidfVectorizer(ngram_range=(1, 2))),
                                       ('clf',
                                        LogisticRegression(C=10,
                                                           random_state=42))]),
             n_jobs=-1, param_grid={}, scoring='precision')

In [12]:
print("Best parameters:", grid_search.best_params_)
best_clf = grid_search.best_estimator_

Best parameters: {}


In [13]:
pred = best_clf.predict(X_test['Review'])
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74      6675
           1       0.95      0.97      0.96     40744

    accuracy                           0.93     47419
   macro avg       0.87      0.84      0.85     47419
weighted avg       0.93      0.93      0.93     47419



## Пример (пайплайн) использования

In [15]:
example = {
    'Review': [
        'Мне понравилась эта обувь, она довольно удобная',
        'Не ожидал такого, отвратительный опыт',
        'В целом неплохо, но есть проблема с качеством материала'
    ]
}

In [16]:
sample = pd.DataFrame.from_dict(example)

In [18]:
sample['Review'] = sample['Review'].apply(preprocess_text)

In [19]:
sample

,Review
0,мне понрав эт обув он довольн удобн
1,эт был сам отвратительн оп в мо жизн
2,в цел неплох но ест проблем с качеств материа


In [20]:
pred_sample = best_clf.predict(sample['Review'])

In [21]:
pred_sample

array([1, 0, 1])